In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import pymc3 as pm
import theano


data = pickle.load(open("../data/filled_data2.pkl", "rb"))

test = data["Page"].unique()


#[page.split(".") in page for page in data["Page"]]
#data["Page"].split(".")


truths = ['EGOIST_ja' in page for page in data["Page"]]
# for page in data["Page"]:
#     page in 'Francisco_el_matemático'
np.array(truths).sum()
for page in data[(truths)]["Page"]:
    print(page)
example = data[(truths)]
example

EGOIST_ja.wikipedia.org_mobile-web_all-agents
EGOIST_ja.wikipedia.org_desktop_all-agents
EGOIST_ja.wikipedia.org_all-access_all-agents
EGOIST_ja.wikipedia.org_all-access_spider


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
55826,EGOIST_ja.wikipedia.org_mobile-web_all-agents,403.0,370.0,383.0,426.0,446.0,413.0,364.0,348.0,337.0,...,690.0,1009.0,1012.0,729.0,688.0,711.0,719.0,668.0,856.0,855.0
84783,EGOIST_ja.wikipedia.org_desktop_all-agents,217.0,189.0,189.0,295.0,254.0,244.0,250.0,218.0,214.0,...,365.0,489.0,424.0,347.0,379.0,416.0,346.0,272.0,458.0,363.0
118732,EGOIST_ja.wikipedia.org_all-access_all-agents,623.0,561.0,576.0,722.0,701.0,660.0,616.0,571.0,554.0,...,1060.0,1505.0,1448.0,1091.0,1087.0,1135.0,1071.0,948.0,1322.0,1224.0
131980,EGOIST_ja.wikipedia.org_all-access_spider,20.0,1.0,3.0,6.0,0.0,1.0,8.0,8.0,4.0,...,43.0,59.0,64.0,41.0,33.0,50.0,44.0,29.0,55.0,46.0


In [2]:
test_set = example.iloc[:,-72:]
train_set = example.iloc[:,:-72]

In [3]:
def add_AR_terms(array, terms_to_add):
    to_return = array.flatten()
    ar = array.flatten()
    for terms in range(terms_to_add):
        ar = np.hstack((np.repeat(np.nan,terms + 1), array.flatten()[:-(abs(terms) +1)]))
#         print(ar)
        to_return = np.column_stack((to_return, ar))
    return to_return

In [4]:
dates = test_set.columns
temp = np.array(test_set.iloc[-1:,:]).reshape(len(dates),1)
test = pd.DataFrame({'ds':list(dates), 'y':[int(item) for item in temp]})

count = 0
for k in train_set["Page"]:
    temp = train_set[train_set["Page"] == k]
    temp = np.array(temp).reshape(len(temp.columns),1)
    non_dated_table = add_AR_terms(temp[1:], 7)
    dates = train_set.columns
    hierarchical_col = [count]* (len(dates) - 1)
    print(k)
    dated_table1 = np.column_stack((dates[1:], hierarchical_col, non_dated_table))
    if count == 0:
        dated_table2 = dated_table1
    else:
        dated_table2 = np.ma.row_stack((dated_table2, dated_table1))
#     train = pd.DataFrame({'ds':list(dates[1:]), 'y':[int(item) for item in temp[1:]]})
    count += 1
hierarchical_df = pd.DataFrame(dated_table2)
hierarchical_df.columns = ["date", "page", "target", "ar1", "ar2", "ar3", "ar4", "ar5", "ar6", "ar7"]
hierarchical_df = hierarchical_df.dropna()
hierarchical_df

EGOIST_ja.wikipedia.org_mobile-web_all-agents
EGOIST_ja.wikipedia.org_desktop_all-agents
EGOIST_ja.wikipedia.org_all-access_all-agents
EGOIST_ja.wikipedia.org_all-access_spider


,date,page,target,ar1,ar2,ar3,ar4,ar5,ar6,ar7
7,2015-07-08,0,348,364,413,446,426,383,370,403
8,2015-07-09,0,337,348,364,413,446,426,383,370
9,2015-07-10,0,326,337,348,364,413,446,426,383
10,2015-07-11,0,353,326,337,348,364,413,446,426
11,2015-07-12,0,397,353,326,337,348,364,413,446
12,2015-07-13,0,305,397,353,326,337,348,364,413
13,2015-07-14,0,325,305,397,353,326,337,348,364
14,2015-07-15,0,296,325,305,397,353,326,337,348
15,2015-07-16,0,790,296,325,305,397,353,326,337
16,2015-07-17,0,546,790,296,325,305,397,353,326


In [5]:
ar1 = theano.shared(np.array(hierarchical_df.iloc[:,3]).astype(int))
ar2 = theano.shared(np.array(hierarchical_df.iloc[:,4:5]).flatten().astype(int))
ar3 = theano.shared(np.array(hierarchical_df.iloc[:,5:6]).flatten().astype(int))
ar4 = theano.shared(np.array(hierarchical_df.iloc[:,6:7]).flatten().astype(int))
ar5 = theano.shared(np.array(hierarchical_df.iloc[:,7:8]).flatten().astype(int))
ar6 = theano.shared(np.array(hierarchical_df.iloc[:,8:9]).flatten().astype(int))
ar7 = theano.shared(np.array(hierarchical_df.iloc[:,9:]).flatten().astype(int))
model_output = theano.shared(np.array(hierarchical_df["target"]).astype(int))
page_idx = theano.shared(np.array(hierarchical_df['page'].values).astype(int))
ar1

<TensorType(int64, vector)>

In [16]:
np.array(hierarchical_df["target"]).astype(int).shape

(2896,)

In [19]:
np.array(hierarchical_df.iloc[:,3:4]).flatten().astype(int).shape

(2896,)

In [21]:
np.array(hierarchical_df['page'].values).astype(int).shape

(2896,)

In [31]:
with pm.Model() as hierarchical_model:
    # Hyperpriors
    mu_a = pm.Normal('mu_alpha', mu=0., sd=10e2)
    sigma_a = pm.HalfCauchy('sigma_alpha', beta=2)
    mu_b = pm.Normal('mu_beta',mu=0., sd = 10e2)
    sigma_b = pm.HalfCauchy('sigma_beta', beta=2)
    
    
    # Intercept for each county, distributed around group mean mu_a
#     a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=len(hierarchical_df['page'].unique()))
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape = 1)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape = 1)
#     b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=len(hierarchical_df['page'].unique()))
    
    # Model error
    eps = pm.Uniform('eps', .01, 10, shape = 1)
    
    # Expected value
#     est = a[page_idx] + b[page_idx] * ar1
    est = a + b * ar1
    #   + b[page_idx] * ar2 + b[page_idx] * ar3 + b[page_idx] * ar4 + b[page_idx] * ar5 + b[page_idx] * ar6 + b[page_idx] * ar7
    
    # Data likelihood
    y = pm.Normal('y', mu=est, sd=eps, observed = model_output)
    
# with hierarchical_model:
#     # zipcode_0 - used for comparison later
#     zip_0 = pm.Lognormal('zip_0', mu=a[0] + b[0], sd=eps)

In [32]:
with hierarchical_model:
    hierarchical_trace = pm.sample(100, init = 'adapt_diag', n_jobs = 1)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [eps, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]
Sampling 2 chains: 100%|██████████| 1200/1200 [00:26<00:00, 45.43draws/s] 
/home/leaferickson/anaconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfftn_numpy(a, s, axes)
There were 5 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9459724758390936, but should be close to 0.8. Try to increase the number of tuning steps.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.

In [15]:
model_output.shape[0]

Subtensor{int64}.0